<a href="https://colab.research.google.com/github/AndersonIM/Detec-o_de_imagem_prova_dio/blob/main/Projeto_Deteccao_de_Imagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Passo 1: Clonar o YOLOv5 e Instalar as Dependências
Contexto: O primeiro passo é baixar o projeto YOLOv5 do GitHub. Em seguida, vamos entrar na pasta do projeto e usar o pip (o gerenciador de pacotes do Python) para instalar automaticamente todas as bibliotecas que ele precisa para funcionar. Note que não há compilação manual aqui.

In [ ]:
# Clona o repositório oficial do YOLOv5
!git clone https://github.com/ultralytics/yolov5

# Entra na pasta que acabamos de clonar
%cd yolov5

# Instala todas as bibliotecas necessárias
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17544, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 17544 (delta 25), reused 3 (delta 3), pack-reused 17502 (from 2)
Receiving objects: 100% (17544/17544), 16.66 MiB | 19.82 MiB/s, done.
Resolving deltas: 100% (12022/12022), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00


Passo 2.1: Baixar e Preparar os Dados Brutos do COCO
Contexto: A célula abaixo fará duas coisas:

Baixar e descompactar: Ela vai baixar o conjunto de imagens de validação do COCO e seus respectivos arquivos de anotações.

Converter os Rótulos: Em seguida, ela executará um script Python para ler os arquivos de anotações, filtrar apenas os objetos person e car, e salvar os rótulos no formato de texto (.txt) que o YOLO entende.

Tudo isso será salvo em uma pasta temporária (/content/val2017/). Organizar esses arquivos na estrutura de pastas que o YOLOv5 exige.

In [ ]:
# Importa bibliotecas necessárias
import os
import json
from tqdm import tqdm
import requests
from zipfile import ZipFile

# ===== 1. BAIXAR E DESCOMPACTAR OS DADOS =====
print("--- Baixando e descompactando dados do COCO (pode demorar alguns minutos)... ---")
%cd /content/

# URLs e caminhos
val_zip_url = 'http://images.cocodataset.org/zips/val2017.zip'
ann_zip_url = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
val_zip_path = 'val2017.zip'
ann_zip_path = 'annotations_trainval2017.zip'

# Baixa os arquivos
if not os.path.exists(val_zip_path):
    r = requests.get(val_zip_url)
    with open(val_zip_path, 'wb') as f:
        f.write(r.content)

if not os.path.exists(ann_zip_path):
    r = requests.get(ann_zip_url)
    with open(ann_zip_path, 'wb') as f:
        f.write(r.content)

# Descompacta
with ZipFile(val_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')
with ZipFile(ann_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Limpa os arquivos .zip
os.remove(val_zip_path)
os.remove(ann_zip_path)
print("--- Download e descompactação concluídos! ---")


# ===== 2. CONVERTER OS RÓTulos PARA O FORMATO YOLO (.txt) =====
print("\n--- Processando e convertendo rótulos... ---")

def convert_coco_to_yolo(coco_data, output_dir, class_map):
    images_info = {img['id']: img for img in coco_data['images']}
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    annotations_by_image = {}
    for ann in coco_data['annotations']:
        cat_id = ann['category_id']
        if cat_id in class_map:
            img_id = ann['image_id']
            if img_id not in annotations_by_image: annotations_by_image[img_id] = []
            annotations_by_image[img_id].append(ann)
    image_paths = []
    for img_id, annotations in tqdm(annotations_by_image.items(), desc="Processando imagens"):
        img_info = images_info[img_id]
        img_w, img_h = img_info['width'], img_info['height']
        label_filename = os.path.splitext(img_info['file_name'])[0] + '.txt'
        label_path = os.path.join(output_dir, label_filename)
        # Salva o caminho completo da imagem para uso posterior
        image_paths.append(os.path.join('/content/val2017', img_info['file_name']))
        with open(label_path, 'w') as f:
            for ann in annotations:
                cat_id = ann['category_id']
                if cat_id in class_map:
                    new_class_id = class_map[cat_id]
                    x, y, w, h = ann['bbox']
                    x_center, y_center, norm_w, norm_h = (x + w / 2) / img_w, (y + h / 2) / img_h, w / img_w, h / img_h
                    f.write(f"{new_class_id} {x_center} {y_center} {norm_w} {norm_h}\n")
    return image_paths

with open('/content/annotations/instances_val2017.json', 'r') as f:
    coco_val = json.load(f)

class_map = {1: 0, 3: 1} # person: 0, car: 1
labels_dir = '/content/val2017'
image_paths_with_objects = convert_coco_to_yolo(coco_val, labels_dir, class_map)

# Salva a lista de imagens filtradas para o próximo passo
with open('filtered_images.txt', 'w') as f:
    for path in image_paths_with_objects:
        f.write(path + '\n')

print(f"\n✅ Passo 2.1 concluído! {len(image_paths_with_objects)} imagens e rótulos foram preparados em uma pasta temporária.")

--- Baixando e descompactando dados do COCO (pode demorar alguns minutos)... ---
/content
--- Download e descompactação concluídos! ---

--- Processando e convertendo rótulos... ---


Processando imagens: 100%|██████████| 2869/2869 [00:00<00:00, 11539.30it/s]


✅ Passo 2.1 concluído! 2869 imagens e rótulos foram preparados em uma pasta temporária.


Passo 2.2: Organizar os Dados na Estrutura de Pastas do YOLOv5
Contexto: O YOLOv5 precisa que os dados sejam separados em pastas de treino e validação. A célula abaixo fará o seguinte:

Criará um novo diretório chamado datasets/coco_person_car.

Dentro dele, criará as subpastas: images/train, images/val, labels/train, e labels/val.

Dividirá aleatoriamente as 2869 imagens que preparamos: 80% para treino e 20% para validação.

Moverá os arquivos de imagem (.jpg) e seus respectivos rótulos (.txt) para as pastas corretas.

In [ ]:
import os
import random
import shutil
from tqdm import tqdm

# Garante que estamos no diretório /content/ para facilitar os caminhos
%cd /content/

print("--- Lendo a lista de imagens filtradas... ---")
# Lê a lista de imagens que preparamos no passo anterior
with open('filtered_images.txt', 'r') as f:
    image_files = [line.strip() for line in f.readlines()]

# Embaralha a lista para garantir uma divisão aleatória
random.shuffle(image_files)

# Define a proporção do split (80% para treino, 20% para validação)
split_ratio = 0.8
split_index = int(len(image_files) * split_ratio)

# Divide a lista em treino e validação
train_files = image_files[:split_index]
val_files = image_files[split_index:]

print("--- Criando a estrutura de pastas do YOLOv5... ---")
# Define os diretórios de destino
base_dir = '/content/datasets/coco_person_car'
train_img_dir = os.path.join(base_dir, 'images/train')
val_img_dir = os.path.join(base_dir, 'images/val')
train_lbl_dir = os.path.join(base_dir, 'labels/train')
val_lbl_dir = os.path.join(base_dir, 'labels/val')

# Cria os diretórios de destino
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# Função para mover os arquivos mostrando uma barra de progresso
def move_files(file_list, img_dest_dir, lbl_dest_dir):
    for img_path in tqdm(file_list, desc=f"Movendo para {os.path.basename(img_dest_dir)}"):
        # O caminho do rótulo é o mesmo da imagem, só muda a extensão
        base_filename = os.path.splitext(os.path.basename(img_path))[0]
        lbl_filename = base_filename + '.txt'
        lbl_path = os.path.join(os.path.dirname(img_path), lbl_filename)

        if os.path.exists(lbl_path):
            # Move o arquivo de imagem
            shutil.move(img_path, img_dest_dir)
            # Move o arquivo de rótulo
            shutil.move(lbl_path, lbl_dest_dir)

# Move os arquivos de treino
print("\n--- Organizando arquivos de TREINO ---")
move_files(train_files, train_img_dir, train_lbl_dir)

# Move os arquivos de validação
print("\n--- Organizando arquivos de VALIDAÇÃO ---")
move_files(val_files, val_img_dir, val_lbl_dir)

print(f"\n✅ Passo 2.2 concluído!")
print(f"Total de imagens: {len(image_files)}")
print(f"Imagens de treino: {len(train_files)}")
print(f"Imagens de validação: {len(val_files)}")
print(f"Dados organizados em: {base_dir}")

/content
--- Lendo a lista de imagens filtradas... ---
--- Criando a estrutura de pastas do YOLOv5... ---

--- Organizando arquivos de TREINO ---


Movendo para train: 100%|██████████| 2295/2295 [00:00<00:00, 6577.31it/s]



--- Organizando arquivos de VALIDAÇÃO ---


Movendo para val: 100%|██████████| 574/574 [00:00<00:00, 6327.58it/s]


✅ Passo 2.2 concluído!
Total de imagens: 2869
Imagens de treino: 2295
Imagens de validação: 574
Dados organizados em: /content/datasets/coco_person_car


In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
%%writefile data/coco_person_car.yaml
# Caminho para as imagens de treino
train: /content/datasets/coco_person_car/images/train

# Caminho para as imagens de validação
val: /content/datasets/coco_person_car/images/val

# Número de classes
nc: 2

# Nomes das classes (na ordem: 0 = person, 1 = car)
names: ['person', 'car']

Writing data/coco_person_car.yaml


Passo 3: Iniciar o Treinamento do YOLOv5
Contexto: A célula abaixo executará o script de treinamento do YOLOv5. Vamos analisar os parâmetros mais importantes que estamos usando:

--img 416: Define o tamanho das imagens para o treinamento (416x416 pixels).

--batch 16: Processará 16 imagens por vez. É um bom valor para a GPU do Colab.

--epochs 50: O modelo "verá" o conjunto de dados de treino completo 50 vezes. Para este projeto, é mais que suficiente para um bom resultado.

--data coco_person_car.yaml: Nosso "mapa" do dataset que acabamos de criar.

--weights yolov5s.pt: O ponto de partida. yolov5s.pt é um modelo "pequeno" (small) do YOLOv5, já pré-treinado no COCO completo. É rápido e ideal para o nosso transfer learning.

In [ ]:
# Garante que estamos na pasta correta
%cd /content/yolov5

# Inicia o treinamento!
!python train.py --img 416 --batch 16 --epochs 50 --data data/coco_person_car.yaml --weights yolov5s.pt

/content/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-08-20 13:34:31.076192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755696871.119512    1457 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755696871.132882    1457 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for 

In [ ]:
# Garante que estamos na pasta correta
%cd /content/yolov5

# Baixa a nova imagem de teste de uma fonte permitida
!wget -O imagem_teste_final.jpg "https://images.pexels.com/photos/3764563/pexels-photo-3764563.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1"

/content/yolov5
--2025-08-20 14:13:09--  https://images.pexels.com/photos/3764563/pexels-photo-3764563.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1
Resolving images.pexels.com (images.pexels.com)... 104.18.66.220, 104.18.67.220, 2606:4700::6812:42dc, ...
Connecting to images.pexels.com (images.pexels.com)|104.18.66.220|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2025-08-20 14:13:10 ERROR 403: Forbidden.

